# Experiment #3: Open vSwitch 4 Networks 2 Bridge Nodes 1 Shared Link

In this experiment we try to connect 4 networks with different hosts together with a bridge in the middle. We will be using Open vSwitch for the bridge, alternate solutions are Linux Bridge and Behavioral Model (BMv2).

## References
- [Open vSwitch Example](https://github.com/fabric-testbed/jupyter-examples/blob/main/fabric_examples/complex_recipes/openvswitch/openvswitch.ipynb)
- [BMv4](https://github.com/p4lang/behavioral-model)

## TODO's

- Add Open vSwitch flow configuratation on ports and so on.
- Add tools on how to track packets in the bridge.
- measure of congestion.


## Topology


<img src="./fig/Testbed-openvswitch-4.png" width=90%>

## Code

In [38]:
from ipaddress import ip_address, IPv4Address, IPv4Network
import ipaddress
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager


fablib = fablib_manager()
fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/alphani/.fabric_token.json
Project ID,220c2d8a-ae05-465a-a98b-8178415e1b30
Bastion Host,bastion.fabric-testbed.net
Bastion Username,sa2994_0000240697
Bastion Private Key File,/home/alphani/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/alphani/work/fabric_config/slice_key.pub
Slice Private Key File,/home/alphani/work/fabric_config/slice_key


In [39]:
slice_name= "Testbed-openvswitch-4"

sites = [site1,site2,site3,site4,site5,site6] = fablib.get_random_sites(count=6)
print(f"Sites: {sites}")

site_node_count = 2
bridge1_name = 'bridge1'
bridge2_name = 'bridge2'

Sites: ['LOSA', 'GPN', 'INDI', 'CERN', 'UTAH', 'UCSD']


In [40]:
slice = fablib.new_slice(name=slice_name)
default_image= 'default_ubuntu_22'

In [41]:
bridge1 = slice.add_node(name=bridge1_name, site=site5, cores=2, ram=8, disk=10, image=default_image, host=f"{site5.lower()}-w1.fabric-testbed.net")
bridge1_nic1 = bridge1.add_component(model='NIC_Basic', name='nic_local_1')
bridge1_nic2 = bridge1.add_component(model='NIC_Basic', name='nic_local_2')
bridge1_nic3 = bridge1.add_component(model='NIC_Basic', name='nic_local_3')

bridge2 = slice.add_node(name=bridge2_name, site=site6, cores=2, ram=8, disk=10, image=default_image, host=f"{site6.lower()}-w1.fabric-testbed.net")
bridge2_nic1 = bridge2.add_component(model='NIC_Basic', name='nic_local_1')
bridge2_nic2 = bridge2.add_component(model='NIC_Basic', name='nic_local_2')
bridge2_nic3 = bridge2.add_component(model='NIC_Basic', name='nic_local_3')

### Connecting bridges nodes 

In [42]:
net = slice.add_l2network(name=f'net-br1-br2')
net.add_interface(bridge1.get_interfaces()[2])
net.add_interface(bridge2.get_interfaces()[2])

### Adding Nodes to bridge 1

In [43]:
for br_num in range(2):
    for site_num in range(2):
        site = sites[br_num*2+site_num]
        print(f"Adding nodes to {site}-bridge{br_num+1}")
        node_name = f"{site.lower()}-bridge{br_num+1}-1"
        node = slice.add_node(name=node_name, site=site, cores=2, ram=8, disk=10, image=default_image, host=f"{site.lower()}-w2.fabric-testbed.net")
        iface = node.add_component(model='NIC_Basic', name='nic_local').get_interfaces()[0]    
        net = slice.add_l2network(name=f"net-br{br_num+1}-{site.lower()}-1")
        net.add_interface(iface)

        if br_num%2==0:
            net.add_interface(bridge1.get_interfaces()[site_num])
        else:
            net.add_interface(bridge2.get_interfaces()[site_num])

Adding nodes to LOSA-bridge1
Adding nodes to GPN-bridge1
Adding nodes to INDI-bridge2
Adding nodes to CERN-bridge2


In [44]:
slice.submit();


Retry: 10, Time: 352 sec


ID,e4fe5cbb-23f0-4049-bf06-9fd23ac92643
Name,Testbed-openvswitch-4
Lease Expiration (UTC),2025-01-12 14:21:56 +0000
Lease Start (UTC),2025-01-11 14:21:56 +0000
Project ID,220c2d8a-ae05-465a-a98b-8178415e1b30
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
cf9c3c57-d267-4c94-b7cb-358a7ecee9ff,bridge1,2,8,10,default_ubuntu_22,qcow2,utah-w1.fabric-testbed.net,UTAH,ubuntu,2001:1948:417:7:f816:3eff:fe75:cf35,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2001:1948:417:7:f816:3eff:fe75:cf35,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
ecaa0b4e-85de-4916-baa3-9d283cfc24c9,bridge2,2,8,10,default_ubuntu_22,qcow2,ucsd-w1.fabric-testbed.net,UCSD,ubuntu,132.249.252.140,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@132.249.252.140,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
30d26560-628d-4046-85c2-01fc8552b201,cern-bridge2-1,2,8,10,default_ubuntu_22,qcow2,cern-w2.fabric-testbed.net,CERN,ubuntu,2001:400:a100:3090:f816:3eff:fe73:188f,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe73:188f,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
c26c62dc-d83f-4cbd-a15b-7650203ab492,gpn-bridge1-1,2,8,10,default_ubuntu_22,qcow2,gpn-w2.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe8a:4c8c,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe8a:4c8c,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
662c70a8-bf19-4323-9d34-5f9a4274fa1d,indi-bridge2-1,2,8,10,default_ubuntu_22,qcow2,indi-w2.fabric-testbed.net,INDI,ubuntu,2001:18e8:fff0:3:f816:3eff:fe01:d2,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2001:18e8:fff0:3:f816:3eff:fe01:d2,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
452d1d7e-c48e-4f5c-a302-2f056df5d9b5,losa-bridge1-1,2,8,10,default_ubuntu_22,qcow2,losa-w2.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:feda:37c1,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:feda:37c1,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
6ad5c020-18fa-4273-b089-66e1b3a8189c,net-br1-br2,L2,L2STS,None,None,None,Active,
8533c006-59cb-4f62-bdb1-d01bc2cce8bb,net-br1-gpn-1,L2,L2STS,None,None,None,Active,
32485656-8607-493e-ade0-52bf0831683c,net-br1-losa-1,L2,L2STS,None,None,None,Active,
d89cafa5-bb0e-4879-a11a-66534a51316e,net-br2-cern-1,L2,L2STS,None,None,None,Active,
00f2b2eb-6f7a-4775-8356-11eedbd9df1a,net-br2-indi-1,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
bridge1-nic_local_2-p1,p1,bridge1,net-br1-losa-1,100,config,,12:09:D4:1A:17:87,enp8s0,enp8s0,fe80::1009:d4ff:fe1a:1787,6,HundredGigE0/0/0/5
bridge1-nic_local_3-p1,p1,bridge1,net-br1-gpn-1,100,config,,12:98:E2:68:0A:58,enp9s0,enp9s0,fe80::1098:e2ff:fe68:a58,6,HundredGigE0/0/0/5
bridge1-nic_local_1-p1,p1,bridge1,net-br1-br2,100,config,,0E:C6:90:95:60:FC,enp7s0,enp7s0,fe80::cc6:90ff:fe95:60fc,6,HundredGigE0/0/0/5
bridge2-nic_local_1-p1,p1,bridge2,net-br2-indi-1,100,config,,36:6D:BC:D1:95:13,enp9s0,enp9s0,fe80::346d:bcff:fed1:9513,6,HundredGigE0/0/0/5
bridge2-nic_local_2-p1,p1,bridge2,net-br2-cern-1,100,config,,2E:62:AE:8A:03:2F,enp7s0,enp7s0,fe80::2c62:aeff:fe8a:32f,6,HundredGigE0/0/0/5
bridge2-nic_local_3-p1,p1,bridge2,net-br1-br2,100,config,,32:7E:B0:E9:FA:FA,enp8s0,enp8s0,fe80::307e:b0ff:fee9:fafa,6,HundredGigE0/0/0/5
losa-bridge1-1-nic_local-p1,p1,losa-bridge1-1,net-br1-losa-1,100,config,,0A:4C:01:D1:D8:41,enp6s0,enp6s0,fe80::84c:1ff:fed1:d841,4,HundredGigE0/0/0/7
gpn-bridge1-1-nic_local-p1,p1,gpn-bridge1-1,net-br1-gpn-1,100,config,,0E:E9:FF:63:55:A6,enp7s0,enp7s0,fe80::ce9:ffff:fe63:55a6,6,HundredGigE0/0/0/7
indi-bridge2-1-nic_local-p1,p1,indi-bridge2-1,net-br2-indi-1,100,config,,06:CC:82:A2:A1:14,enp7s0,enp7s0,fe80::4cc:82ff:fea2:a114,4,HundredGigE0/0/0/9
cern-bridge2-1-nic_local-p1,p1,cern-bridge2-1,net-br2-cern-1,100,config,,0A:F6:CB:37:F0:A4,enp7s0,enp7s0,fe80::8f6:cbff:fe37:f0a4,4,HundredGigE0/0/0/7



Time to print interfaces 390 seconds


# OVS 

In [45]:
try:
    for node in slice.get_nodes():
        if node.get_name().startswith("bridge"):
            stdout, stderr = node.execute('yes | sudo apt-get -y update && sudo apt-get upgrade', quiet=True) 
            stdout, stderr = node.execute('yes | sudo apt-get -y install openvswitch-switch openvswitch-common', quiet=True)
            stdout, stderr = node.execute('sudo apt-get -y install net-tools', quiet=True)
            print(f"done bridge: {node.get_name()}")
    print("Done")
except Exception as e:
    print(f"Exception: {e}")

done bridge: bridge1
done bridge: bridge2
Done


In [46]:
bridge1 = slice.get_node(name=bridge1_name)
stdout, stderr = bridge1.execute('sudo ovs-vsctl add-br br0')
for interface in bridge1.get_interfaces():
    stdout, stderr = bridge1.execute(f'sudo ovs-vsctl add-port br0 {interface.get_physical_os_interface_name()}')
    #Remove IP addresses for all interfaces
    stdout, stderr = bridge1.execute(f'sudo ifconfig {interface.get_physical_os_interface_name()} 0')
    
#bring the bridge up
stdout, stderr = bridge1.execute('sudo ifconfig br0 up')

print("Done")
stdout, stderr = bridge1.execute('sudo ovs-vsctl set bridge br0 stp_enable=true')
stdout, stderr = bridge1.execute('sudo ovs-appctl stp/show')


Done
---- br0 ----
Root ID:
  stp-priority  32768
  stp-system-id   72:fa:fa:c8:49:4c
  stp-hello-time  2s
  stp-max-age     20s
  stp-fwd-delay   15s
  This bridge is the root

Bridge ID:
  stp-priority  32768
  stp-system-id   72:fa:fa:c8:49:4c
  stp-hello-time  2s
  stp-max-age     20s
  stp-fwd-delay   15s

  Interface  Role       State      Cost  Pri.Nbr 
  ---------- ---------- ---------- ----- -------
  enp7s0     designated listening  2     128.1
  enp8s0     designated listening  2     128.2
  enp9s0     designated listening  2     128.3



In [47]:
bridge2 = slice.get_node(name=bridge2_name)
stdout, stderr = bridge2.execute('sudo ovs-vsctl add-br br0')
for interface in bridge2.get_interfaces():
    stdout, stderr = bridge2.execute(f'sudo ovs-vsctl add-port br0 {interface.get_physical_os_interface_name()}')
    #Remove IP addresses for all interfaces
    stdout, stderr = bridge2.execute(f'sudo ifconfig {interface.get_physical_os_interface_name()} 0')
    
#bring the bridge up
stdout, stderr = bridge2.execute('sudo ifconfig br0 up')

print("Done")
stdout, stderr = bridge2.execute('sudo ovs-vsctl set bridge br0 stp_enable=true')
stdout, stderr = bridge2.execute('sudo ovs-appctl stp/show')


Done
---- br0 ----
Root ID:
  stp-priority  32768
  stp-system-id   0a:79:71:48:23:46
  stp-hello-time  2s
  stp-max-age     20s
  stp-fwd-delay   15s
  This bridge is the root

Bridge ID:
  stp-priority  32768
  stp-system-id   0a:79:71:48:23:46
  stp-hello-time  2s
  stp-max-age     20s
  stp-fwd-delay   15s

  Interface  Role       State      Cost  Pri.Nbr 
  ---------- ---------- ---------- ----- -------
  enp8s0     designated listening  2     128.1
  enp7s0     designated listening  2     128.2
  enp9s0     designated listening  2     128.3



# Host Setup 

In [48]:
networks = [
    "10.10.10.1/24",
    "10.10.10.2/24",
    "10.10.10.3/24",
    "10.10.10.4/24"
]

for br_num in range(2):
    for i in range(2):
        site = sites[br_num*2 + i]
        host = slice.get_node(name=f'{site.lower()}-bridge{br_num+1}-1')
        stdout, stderr = host.execute('sudo apt-get -y install net-tools', quiet=True)
        stdout, stderr = host.execute(f'sudo ip link set dev {host.get_interfaces()[0].get_physical_os_interface_name()} up', quiet=True)
        stdout, stderr = host.execute(f'sudo ip addr add {networks[i+2*br_num]} dev {host.get_interfaces()[0].get_physical_os_interface_name()}', quiet=True)


In [49]:
host1 = slice.get_node(name=f'{site1.lower()}-bridge1-1')
stdout, stderr = host1.execute('ping 10.10.10.2 -c 5')

PING 10.10.10.2 (10.10.10.2) 56(84) bytes of data.
64 bytes from 10.10.10.2: icmp_seq=1 ttl=64 time=71.7 ms
64 bytes from 10.10.10.2: icmp_seq=2 ttl=64 time=35.6 ms
64 bytes from 10.10.10.2: icmp_seq=3 ttl=64 time=35.6 ms
64 bytes from 10.10.10.2: icmp_seq=4 ttl=64 time=35.6 ms
64 bytes from 10.10.10.2: icmp_seq=5 ttl=64 time=35.6 ms

--- 10.10.10.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4007ms
rtt min/avg/max/mdev = 35.572/42.800/71.655/14.427 ms


In [50]:
stdout, stderr = host1.execute('ping 10.10.10.3 -c 5')

PING 10.10.10.3 (10.10.10.3) 56(84) bytes of data.
64 bytes from 10.10.10.3: icmp_seq=1 ttl=64 time=155 ms
64 bytes from 10.10.10.3: icmp_seq=2 ttl=64 time=77.2 ms
64 bytes from 10.10.10.3: icmp_seq=3 ttl=64 time=77.2 ms
64 bytes from 10.10.10.3: icmp_seq=4 ttl=64 time=77.1 ms
64 bytes from 10.10.10.3: icmp_seq=5 ttl=64 time=77.2 ms

--- 10.10.10.3 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4004ms
rtt min/avg/max/mdev = 77.147/92.739/155.016/31.138 ms


In [51]:
stdout, stderr = host1.execute('ping 10.10.10.4 -c 5')

PING 10.10.10.4 (10.10.10.4) 56(84) bytes of data.
64 bytes from 10.10.10.4: icmp_seq=1 ttl=64 time=373 ms
64 bytes from 10.10.10.4: icmp_seq=2 ttl=64 time=186 ms
64 bytes from 10.10.10.4: icmp_seq=3 ttl=64 time=186 ms
64 bytes from 10.10.10.4: icmp_seq=4 ttl=64 time=186 ms
64 bytes from 10.10.10.4: icmp_seq=5 ttl=64 time=186 ms

--- 10.10.10.4 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4001ms
rtt min/avg/max/mdev = 186.330/223.778/373.420/74.820 ms


In [37]:
slice.delete()